## Prompt Engineering in LangSmith

### Import environment variables

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

### Pull in Prompt from Prompthub

In [ ]:
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate
from langsmith.utils import LangSmithNotFoundError

client = Client(api_key=LANGSMITH_API_KEY)

# Define the prompt template
eli5_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at explaining complex topics in simple terms that a 5-year-old could understand. 

Your task is to take a complex question and context information, then provide a clear, simple explanation using:
- Simple words and concepts
- Analogies and examples from everyday life
- Short sentences
- Engaging and friendly tone

Keep your explanation concise but complete.

Question: {question}

Context: {context}

Please explain this in simple terms that a 5-year-old would understand:
""")])

# Try to pull the prompt, if it doesn't exist, push it first
try:
    print("Trying to pull existing prompt...")
    prompt = client.pull_prompt("eli5-concise", include_model=True)
    print("✅ Successfully pulled existing prompt from LangSmith")
except LangSmithNotFoundError:
    print("❌ Prompt not found. Creating and pushing new prompt...")
    
    # Push the prompt to LangSmith
    client.push_prompt(
        "eli5-concise",
        object=eli5_prompt_template,
        description="A prompt for explaining complex topics in simple terms that a 5-year-old could understand"
    )
    print("✅ Successfully pushed prompt to LangSmith")
    
    # Now pull the prompt back
    prompt = client.pull_prompt("eli5-concise", include_model=True)
    print("✅ Successfully pulled the newly created prompt")

### Setup AI Application

Let's first setup our web search tool, as usual

In [ ]:
# Initialize web search tool

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=1)

Let's now create our application, same as in the tracing module. This time, our prompt is the one pulled from PromptHub

In [11]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

# Create Application
openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in web_docs])
    return web_results
    
@traceable
def explain(question, context):
    # Format the prompt with the question and context
    messages = prompt.format_messages(question=question, context=context)
    
    # Call the OpenAI API with the formatted messages
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": msg.type, "content": msg.content} for msg in messages],
        temperature=0.7
    )
    
    return response.choices[0].message.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer

### Test Application

In [ ]:
question = "what is complexity economics?"
print(eli5(question))